Assista estas aulas para a teoria base como complemento:
- https://www.youtube.com/watch?v=NSrOa4qnhvk
- https://www.youtube.com/watch?v=2vSePCL156w
- https://www.youtube.com/watch?v=gRb4OnpeoEg


In [ ]:
# Carregando as bibliotecas

import pandas as pd
import numpy as np
import yfinance as yf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from math import sqrt

from statsmodels.tsa.stattools import adfuller

import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from statsmodels.regression.rolling import RollingOLS
from datetime import datetime

get_ipython().run_line_magic("matplotlib", "inline")
import warnings
warnings.filterwarnings("ignore")

In [ ]:
start = "2022-06-30"
end = "2022-10-31"
#end = datetime.today().strftime("%Y-%m-%d")

tickers = ["BOVA11.SA", "PETR4.SA", "VALE", "ITUB4.SA"
           , "CSNA3.SA", "BBAS3.SA", "WEGE3.SA", "B3SA3.SA" ,  "JBSS3.SA"
           , "ITSA4.SA", "CMIG4.SA", "SBSP3.SA", "CCRO3.SA", "UGPA3.SA"
           , "MULT3.SA", "MGLU3.SA", "EQTL3.SA", "LREN3.SA", "ENBR3.SA"
           , "ABEV3.SA", "SUZB3.SA", "BBDC4.SA", "BBSE3.SA", "ECOR3.SA"
           , "RADL3.SA"]

ativos = []
for ticker in tickers:
    try:
        print(ticker)
        cotacoes = yf.download(ticker, start, end)["Adj Close"]
        cotacoes = pd.DataFrame(cotacoes)
        cotacoes.columns = [ticker]
        ativos.append(cotacoes)
    except:
        pass
    base_ativos = pd.concat(ativos, axis = 1)
base_ativos.sort_index(inplace = True)
base_ativos.head()
base_ativos.dropna(inplace = True)

In [ ]:
base_ativos

In [ ]:
# Função para encontrar pares cointegrados

def encontra_pares(base):
    n = base.shape[1]
    pvalor_matriz = np.ones((n, n))
    keys = base.keys()
    pares = []
    for i in range(n):
        for j in range(i+1, n):
            a1 = base[keys[i]]
            a2 = base[keys[j]]
            resultado = coint(a1, a2)
            score = resultado[0]
            pvalor = resultado[1]
            pvalor_matriz[i, j] = pvalor
            if pvalor < 0.05:
                pares.append((keys[i], keys[j]))
    return pvalor_matriz, pares

In [ ]:
# Encontremos então, os pares

pvalores, pares = encontra_pares(base_ativos)

In [ ]:
pares

In [ ]:
pares[25]

In [ ]:
a1 = pares[25][0]
a2 = pares[25][1]

In [ ]:
x_train = sm.add_constant(base_ativos[a1])
y_train = base_ativos[a2]
regressao = sm.OLS(y_train, x_train).fit()

print(regressao.params)
print("-----------------------------")
print(regressao.params[a1])

beta = regressao.params[a1]

In [ ]:
# Esse beta preferencialmente tem que ser positivo...

In [ ]:
# Normaliza o resíduo pelo z-score

residuo = y_train - (regressao.params[0] + beta*x_train[a1])


def normaliza_zscore(x):
    return (x - x.mean())/np.std(x)


residuo_norm = normaliza_zscore(residuo)

In [ ]:
# Atenção: esta parte não está presente na aula gravada e precisa ser considerada

res_mean = residuo.mean()
res_std = np.std(residuo)

In [ ]:
residuo_norm

In [ ]:
#Visualiza o z-score

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = residuo_norm.index
                         , y = residuo_norm
                         , name = "Z Score"
                         ,line = dict(color = "black"))
              , row = 1, col = 1)

fig.add_hline(y = 2, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = 1, line_width = 1, line_dash = "dash", line_color = "green")

fig.add_hline(y = -2, line_width = 1, line_dash = "dash", line_color = "red")
fig.add_hline(y = -1, line_width = 1, line_dash = "dash", line_color = "red")


fig.update_layout(height = 600, width = 800
                  , title_text = "Z-Score Long & Short por cointegração " + a1 + " - "+ a2
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Tempo"
                  , yaxis_title = "Z-Score"
                  , showlegend = False
                  , legend_title = "Carteira"
                  , font = dict(size = 13, color = "Black")
                 )
fig.update_layout(hovermode = "x")

fig.show()

In [ ]:
# Mas estes resíduos são estacionários?

adf_diff1 = adfuller(residuo_norm)
print("ADF Test Statistic: %.2f" % adf_diff1[0])
print("5%% Critical Value: %.2f" % adf_diff1[4]['5%'])
print("p-value: %.2f" % adf_diff1[1])

In [ ]:
# Vamos pegar a base recente

start = "2022-11-01"
end = datetime.today().strftime("%Y-%m-%d")

tickers = [a1, a2]

ativos_ls = []
for ticker in tickers:
    try:
        print(ticker)
        cotacoes = yf.download(ticker, start, end)["Adj Close"]
        cotacoes = pd.DataFrame(cotacoes)
        cotacoes.columns = [ticker]
        ativos_ls.append(cotacoes)
    except:
        pass
    base_ls = pd.concat(ativos_ls, axis = 1)
base_ls.sort_index(inplace = True)
base_ls.head()
base_ls.dropna(inplace = True)

In [ ]:
# Recalcular os resíduos
# Atenção, pequena modificação em relação ao código da aula gravada

residuo_ls = base_ls[a2] - (regressao.params[0] + beta*base_ls[a1])


def normaliza_zscore_ls(x):
    return (x - res_mean)/res_std


residuo_norm_ls = normaliza_zscore_ls(residuo_ls)

In [ ]:
# Agora vamos avaliar os resultados
# Atenção, pequena modificação em relação ao código da aula gravada

resultados = pd.concat([residuo_norm_ls, residuo_ls
                        , base_ls[a1].pct_change()
                       ,base_ls[a2].pct_change()], axis = 1)
resultados.columns = ["ZScore", "Residuo", a1, a2]

# Vamos fazer o trade com 1 desvio padrão
dev = 1.2


# Resíduo = a2 - (beta0 + beta*a1)
# Se o resíduo é negativo, você vai long no spread
# Significa que a1 é maior do que a2, então você vende a1 e compra a2
# Em qual proporção? No beta. 1 ação de a2 para beta (X) ações de a1
# Detalhes aqui: https://www.youtube.com/watch?v=2vSePCL156w&t=238s

# Se o resíduo for positivo, você vai short no spread
# Sifnigica que a2 é maior do que a1, então você compra ai e vende a2

resultados["sinal"] = 0

resultados["sinal"] = np.where(resultados["ZScore"] < -dev, (resultados[a2] - beta*resultados[a1])
                              , np.where(resultados["ZScore"] > dev, (-resultados[a2] + beta*resultados[a1])
                                        , 0))

resultados["retornos_acc"] = resultados["sinal"].cumsum()

In [ ]:
resultados

In [ ]:
#Visualiza o resultado do L&S por cointegração

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = resultados.index
                         , y = resultados["retornos_acc"]*100
                         , name = "Z Score"
                         ,line = dict(color = "black"))
              , row = 1, col = 1)

fig.update_layout(height = 600, width = 800
                  , title_text = "Resultado - Long & Short por cointegração " + a1 + " - "+ a2
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Tempo"
                  , yaxis_title = "Retorno (%)"
                  , showlegend = False
                  , legend_title = "Carteira"
                  , font = dict(size = 13, color = "Black")
                 )
fig.update_layout(hovermode = "x unified")

fig.show()